In [13]:
import pandas as pd
import numpy as np
import win32com.client as win32


# Carregar o arquivo Excel (inventário)
inventario = pd.read_excel('D:\Youtube\Videos análise de dados\Análise de estoque com envio de e-mail\Inventário.xlsx')

# Carregar o arquivo Excel (DADOS SAP)
sap = pd.read_excel('D:\Youtube\Videos análise de dados\Análise de estoque com envio de e-mail\Tabela SAP.xlsx')


# Exibir o conteúdo das planilhas
print(inventario)

print(sap)



        QNOTE       PN  QUANTIDADE ESTOQUE    LOCAL
0   218012345  ZYX3456           5    G800  F01 C01
1   218012346  ZYX3457           6    G140  F01 C02
2   218012347  ZYX3458           7    G800  F01 C03
3   218012348  ZYX3459           8    G140  F01 C04
4   218012349  ZYX3460           9    G800  F01 C05
5   218012350  ZYX3461          10    G140  F01 C06
6   218012351  ZYX3462          11    G800  F01 C07
7   218012352  ZYX3463          12    G140  F01 C08
8   218012353  ZYX3464          13    G800  F01 C09
9   218012354  ZYX3465          14    G140  F01 C10
10  218012355  ZYX3466          15    G800  F01 C11
11  218012356  ZYX3467          16    G140  F01 C12
12  218012357  ZYX3468          17    G800  F01 C13
13  218012358  ZYX3469          18    G140  F01 C14
14  218012359  ZYX3470          19    G800  F01 C15
15  218012360  ZYX3471          20    G140  F01 C16
16  218012361  ZYX3472          21    G800  F01 C17
17  218012362  ZYX3473          22    G140  F01 C18
18  21801236

In [14]:
# Fazer o merge usando as colunas 'PN' (Inventário) e 'Nº do Material' (SAP)
merged = pd.merge(inventario, sap, left_on='PN', right_on='Nº do Material', how='outer')

# Calcular a diferença de quantidades usando o maior valor menos o menor valor
merged['Divergência'] = np.maximum(merged['QUANTIDADE'], merged['Blocked Stock']) - np.minimum(merged['QUANTIDADE'], merged['Blocked Stock'])

# Filtrar as divergências diferentes de zero
divergencias = merged[merged['Divergência'] != 0]

# Configurar os detalhes do e-mail
remetente = 'seu_email@outlook.com'
destinatario = 'supervisor@example.com'
assunto = 'Divergências Encontradas'

# Construir a mensagem de e-mail
mensagem = f'Divergências encontradas:\n\n'
for _, linha in divergencias.iterrows():
    pn = linha['PN']
    quantidade_inventario = linha['QUANTIDADE']
    quantidade_sap = linha['Blocked Stock']
    divergencia = linha['Divergência']
    mensagem += f'PN: {pn}\nQuantidade no inventário: {quantidade_inventario}\nQuantidade no SAP: {quantidade_sap}\nDivergência: {divergencia} peças\n\n'

# Criar uma instância da aplicação Outlook
outlook = win32.Dispatch('Outlook.Application')

# Criar um e-mail
email = outlook.CreateItem(0)
email.Subject = assunto
email.Body = mensagem
email.To = destinatario
email.SentOnBehalfOfName = remetente

# Enviar o e-mail
try:
    email.Send()
    print('E-mail enviado com sucesso!')
except Exception as e:
    print(f'Erro ao enviar o e-mail: {str(e)}')

E-mail enviado com sucesso!
